In [2]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import gc
import os
# import numpy as np
from skimage import io, color, filters, measure, exposure, feature
from shapely.geometry import MultiPoint, Polygon
import shapely
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.collections import PatchCollection
import numpy as np


In [3]:

#reproduce cells
def reproduce(occupations, active_cell_counts, age_of_first_occupations, t, alpha, dT, gamma, delta, beta, senesc_rate, h, contact_inhib_thresh, prob_diff):
    for n in range(N): #all of these should have occupation number 1 or 0 after the last adjustment step
        if occupations[n] == 1:
            if active_cell_counts[n] == 1: #is the cell able to actively reproduce?
                if np.random.rand() < senesc_rate*dT:
                    active_cell_counts[n] = 0 #render the cell inactive if we're trialling senescence
                else:
                    occupied_neighbour_no = len(list(filter(lambda i: (occupations[i]==1), neighbours[n]))) #how many neighbours does a cell have?
                    if np.random.rand() < (alpha + beta/(1+ np.exp(-h*(contact_inhib_thresh-occupied_neighbour_no))))*dT: #compute division probability
                        occupations[n] = 2 #cell is double-occupied
                        if np.random.rand() < prob_diff: #is this new cell active?
                            active_cell_counts[n] = 2 
    return occupations, active_cell_counts, age_of_first_occupations


#adjust positions until we have no double occupations
#rule: if a cell becomes double-occupied through, then mark which cell was just added to it
def shift(occupations, active_cell_counts, age_of_first_occupations, type_just_added, t, alpha, dT, gamma, delta, beta, senesc_rate, h, contact_inhib_thresh, prob_diff):
    keep_adjusting = True
    while keep_adjusting: 
        any_double_occupied = False #assume none are double occupied
        for n in range(N): #run through every point in space
            if occupations[n] > 1: #is it double-occupied? 
                any_double_occupied = True
                empty_neighbours = list(filter(lambda i: (occupations[i]==0), neighbours[n])) #are there any empty neighbours?
                if len(empty_neighbours)==0: #if all neighbours are occupied
                    ps = [np.exp(-gamma*occupations[i]) for i in neighbours[n]] #weight by their occupations
                    norm = sum(ps) #normalise
                    ps = [p/norm for p in ps]
                    index = np.random.choice(neighbours[n], size=1, p = ps)[0] #choose one to move to at random
                    occupations[index] += 1 #move it to that lattice-point
                    #are we moving an active cell or an inactive cell?
                    if type_just_added[n] == 1: #active cell was just moved there so choose from between other types
                        prob_active = (active_cell_counts[n]-1)/(occupations[n]-1) #fraction of the remaining cells which are active
                        if np.random.rand() < prob_active:
                            active_cell_counts[n] -= 1 #it is active
                            active_cell_counts[index] += 1 #so move it
                            type_just_added[index] = 1 #type just moved was active
                        else:
                            type_just_added[index] = -1 #we just moved an inactive cell
                    else:
                        if type_just_added[n] == -1:
                            prob_active = (active_cell_counts[n])/(occupations[n]-1) #fraction of the remaining cells which are active (retain an inactive cell)
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1 #we just moved an active cell here
                            else:
                                type_just_added[index] = -1 #we just moved an inactive cell here
                        else: #we have no information about what was just added
                            prob_active = active_cell_counts[n]/occupations[n] #what fraction of these cells are active?
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1 
                            else:
                                type_just_added[index] = -1
                else:
                    contact_list = [len(list(filter(lambda i: (occupations[i]>0), neighbours[m]))) for m in empty_neighbours] #number of neighbours possessed by every neighbouring empty lattice-point
                    weights = [np.exp(delta*c) for c in contact_list] #weight by number of neighbours
                    norm = sum(weights)
                    ps = [w/norm for w in weights]
                    index = np.random.choice(empty_neighbours, size=1, p=ps)[0]
                    occupations[index] += 1 #move to an empty one at random
                    age_of_first_occupations[index] = t
                    if type_just_added[n] == 1: #active cell was just moved there so choose from between other types
                        prob_active = (active_cell_counts[n]-1)/(occupations[n]-1)
                        if np.random.rand() < prob_active:
                            active_cell_counts[n] -= 1 #it is active
                            active_cell_counts[index] += 1 #so move it
                            type_just_added[index] = 1
                        else:
                            type_just_added[index] = -1 #moving an inactive cell
                    else:
                        if type_just_added[n] == -1:
                            prob_active = (active_cell_counts[n])/(occupations[n]-1)
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1
                            else:
                                type_just_added[index] = -1
                        else: #no information about what was just added
                            prob_active = active_cell_counts[n]/occupations[n]
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1
                            else:
                                type_just_added[index] = -1
                 #   print("jammed")
                occupations[n] -= 1 #take a cell away from the original cell
        keep_adjusting = any_double_occupied #have we found any double-occupied?
    return occupations, active_cell_counts, type_just_added, age_of_first_occupations

In [4]:
#now run this once
#choose centre roughly

n = 30 #number of cells in each direction
N = n**3 #number of cells total


v_0 = 80 #volume of one cell
r_0 = np.cbrt(v_0)


xs, ys, zs = np.zeros(N), np.zeros(N), np.zeros(N)

overall_neighbour_list = []

#make a list of neighbours
for i in range(n):
    for j in range(n):
        for k in range(n):
            neighbour_list = []
            index = i*(n**2) + j*n + k
            xs[index], ys[index], zs[index] = i*r_0, j*r_0, k*r_0
            for i_ in range(i-1, i+2):
                for j_ in range(j-1, j+2):
                    for k_ in range(k-1, k+2):
                        new_index = i_*(n**2) + j_*n + k_
                        if (0 <= i_ < n) and (0 <= j_ < n) and (0 <= k_ < n): 
                            if 0 <= new_index < N and new_index != index:
                                neighbour_list.append(new_index)
            overall_neighbour_list.append(np.array(neighbour_list))
            
neighbours= overall_neighbour_list

def run_simulation(alpha=0.0, dT=0.01, gamma=10, delta=1.0, beta=0.0, senesc_rate=0.0, h=1, contact_inhib_thresh=0, prob_diff=1, max_cells=5000):
    t = 0

    age_of_first_occupations = np.zeros(N) #keep track of the time at which a lattice point was first occupiedimport numpy as np

    centre = (n**2)*int(n/2) + (n*int(n/2)) + int(n/2) #set 

    occupations, active_cell_counts, type_just_added = np.zeros(N), np.zeros(N), np.zeros(N) #set things up- one occupied cell in the centre
    occupations[centre], active_cell_counts[centre], age_of_first_occupations[centre], type_just_added[centre] = 1, 1, 0, 1


    no_cells = 1

    pops = [] #track number of cells
    overall_radii = [] #track the maximum radius

    while no_cells < max_cells and sum(active_cell_counts)>0:
        occupations, active_cell_counts, age_of_first_occupations = reproduce(occupations, active_cell_counts, age_of_first_occupations, t, alpha, dT, gamma, delta, beta, senesc_rate, h, contact_inhib_thresh, prob_diff)
     #   print("reproduction")
        occupations, active_cell_counts, type_just_added, age_of_first_occupations = shift(occupations, active_cell_counts, age_of_first_occupations, type_just_added, t, alpha, dT, gamma, delta, beta, senesc_rate, h, contact_inhib_thresh, prob_diff)
     #   print("shift")
        no_cells = sum(occupations)
        occupied = list(filter(lambda i: (occupations[i]==1), np.arange(N)))
        radii = [np.sqrt((xs[i]-xs[centre])**2 + (ys[i]-ys[centre])**2) for i in occupied]
        overall_radii.append(max(max(radii), r_0))
        #print(no_cells)
        t += dT
        pops.append(no_cells)

    return occupations, t

In [5]:
def plot(name, occupations, elev, azim):
    to_plot = list(filter(lambda i: (occupations[i]==1), np.arange(N))) #find all occupied cells
    xs_to_plot, ys_to_plot, zs_to_plot = [xs[i] for i in to_plot], [ys[i] for i in to_plot], [zs[i] for i in to_plot]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    p = ax.scatter(xs_to_plot, ys_to_plot, zs_to_plot)
    ax.view_init(elev=elev, azim=azim)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    # make the grid lines transparent
    ax.xaxis._axinfo["grid"]['color'] = (1, 1, 1, 0)
    ax.yaxis._axinfo["grid"]['color'] = (1, 1, 1, 0)
    ax.zaxis._axinfo["grid"]['color'] = (1, 1, 1, 0)
    ax.set_axis_off()
    fig.tight_layout()
    fig.savefig(name+"_"+str(elev)+"_"+str(azim)+".png", format="png", dpi=100)
    #plt.show()
    plt.cla()
    plt.clf()
    plt.close('all')
    gc.collect()
    

In [6]:
def run_plot_save(name, alpha=0.0, dT=0.01, gamma=10, delta=1.0, beta=0.0, senesc_rate=0.0, h=1, contact_inhib_thresh=0, prob_diff=1, max_cells=5000):
    occupations, t = run_simulation(alpha=alpha, dT=dT, gamma=gamma, delta=delta, beta=beta, senesc_rate=senesc_rate, h=h, contact_inhib_thresh=contact_inhib_thresh, prob_diff=prob_diff, max_cells=max_cells)
    np.save(name+"_occupations.npy", occupations)
    np.save(name+"_params_time.npy", [alpha, dT, gamma, delta, beta, senesc_rate, h, contact_inhib_thresh, prob_diff, t])
    for elev in np.arange(-60, 60, 40):
        for azim in np.arange(-60, 60, 40): #cover all possible outlines, i.e. range over one half-surface
            plot(name, occupations, elev, azim)
    

In [7]:


def measure_curv(name, thresh_coeff=0.8):


    # Step 2: Load and preprocess the image
    image = io.imread(name)
    rgb_image = color.rgba2rgb(image)
    grayscale_image = color.rgb2gray(rgb_image)
    blurred_image = filters.gaussian(grayscale_image, sigma=10)
    threshold_value = thresh_coeff*filters.threshold_otsu(blurred_image)
    binary_image = grayscale_image > threshold_value


    # Step 3: Apply edge detection (optional)
    edges = feature.canny(binary_image)

    # Step 4: Find contours
    contours = measure.find_contours(edges, level=0.5)
    
    # Plot the original image
    #fig, ax = plt.subplots()
    #ax.imshow(rgb_image)

    # Plot contours over the original image
    #for contour in contours:
    #    ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color='red')

    #largest_contour = max(contours, key=lambda x: len(x))
    #ax.plot(largest_contour[:, 1], largest_contour[:, 0], linewidth=2, color='yellow')

    # Concatenate all contours into a single array
    points = np.concatenate(contours)

    # Step 5: Compute concave hull
    multi_point = MultiPoint(points)
    hull = shapely.concave_hull(multi_point, ratio=0.05)


    # Plot the concave hull as a polygon
    ys, xs = hull.exterior.xy
    circularity = 4*np.pi*hull.area/(hull.length**2)
    #ax.plot(xs, ys, linewidth=2, color='green')

    #plt.show()
    del [image, rgb_image, grayscale_image, blurred_image, threshold_value, binary_image]
    print(circularity)
    return circularity


In [7]:
c_curv = np.zeros((16, 10, 3, 3))



#first, vary c
for c_, c in enumerate(range(10, 26)):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
                try:
                    curv = measure_curv("c_varying/c_varying_"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    c_curv[c_][r_][e_][a_] = curv
                    print(c, r, elev, azim)
                except:
                    name = "c_varying/c_varying_"+str(c)+"_"+str(r)
                    occupations = np.load("c_varying/c_varying_"+str(c)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("c_varying/c_varying_"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    c_curv[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.path.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")
    
    
np.save("curvature_matrix_c.npy", c_curv)

0.6227273796885334
0.6356032862953769
0.5965638851078956
0.7053773738464004
0.5761037460909589
0.5894402708868158
0.6241783303457046
0.6945402026790295
0.5880615786571051
0.6534988050369731
0.642764428133136
0.6181733059364044
0.4993536626239757
0.6095727044092172
0.7156484306986113
0.5995898659360567
0.6549985694737136
0.6385979076377971
0.5558872657798357
0.5447227708521236
0.5592156668043886
0.5299148409886418
0.5012596248535374
0.5113464459944406
0.524575049316757
0.5377368746888352
0.5369023338121641
0.5456133935562212
0.49414408904490803
0.47179984241985573
0.613474825282817
0.5140899444519011
0.4895311231236747
0.7015790990918165
0.5733937623027783
0.6451369145932184
0.5963129976724024
0.5962184565645408
0.569825680837317
0.6434387869002034
0.5815663104363039
0.6074633823242233
0.49582449305767684
0.5193279224569121
0.646079393153402
0.5831421112851569
0.5569334706183144
0.5123053765552604
0.5732902532523113
0.6181035603861501
0.5871724790514604
0.5344008064281244
0.660160126985

0.7614299013434647
0.7999180267067032
0.8560644860762711
0.8268613335836071
0.7918805625058009
0.7908400844982192
0.8168416771164201
0.7879604881793695
0.7955902417782321
0.8088150583400939
0.7615011111140261
0.791511216256993
0.7689470951986886
0.7948732202657889
0.8152490311148001
0.7142416806012369
0.7901404399891673
0.7989929422361537
0.7989756961167968
0.8138868888809778
Repeat 9 completed.
0.8361316628869849
0.7943030842864185
0.8190705586759863
0.8361452737944962
0.8426525745783743
0.7847485255050126
0.8559375833027996
0.7994992325957369
0.8164514624846065
0.8181005237824425
0.837470933918348
0.8395315528780809
0.8371090411779382
0.830322140906191
0.7896612158072307
0.8185040625376578
0.8153105591531347
0.8162235126826247
0.7969301081002517
0.8242680964789686
0.8100041810073753
0.8412108749060432
0.793695271179925
0.7952371828006076
0.7916624265958578
0.8357523002449269
0.7928453237802345
0.7460045907849178
0.773596411644495
0.765072105417303
0.7315016371740093
0.768091259249106

0.799570094586899
0.8284845986638353
0.8366378879091917
0.8188320112107405
0.7963239349595953
0.8760085797968054
0.8761908091393935
0.8576157311060132
0.8232249960796284
0.8670906426645082
0.8304963403000697
0.8592252518509308
0.8457252795438644
0.826119051608364
0.8626562744988033
0.8709412625113222
0.8914099017886853
0.8417148581099162
0.8488462765211283
0.8677494565103163
0.8166935128102351
0.8130566824115617
0.817415529372061
0.8353545206693042
0.8545274199160902
0.8754176158178248
0.8114225656989208
0.8272798561043451
0.8220012239022132
0.847878511646835
0.8374952304281085
0.8394166841364935
0.8842855637642792
0.8547912827865888
0.8457329048350527
0.8523729130054056
0.87785128446317
0.8523337690361923
0.8790754818131644
0.8706199387098501
0.8640931742827085
Repeat 9 completed.
0.8966441878918946
0.8882249547271922
0.8954718857461132
0.8587189704121685
0.8505876410315314
0.862827828704294
0.8678176392054534
0.8411334863478335
0.8484035208375632
0.8353603935427708
0.8472101317265348

0.8465970525620895
0.8703398001437707
0.8744223562649722
0.849908461200688
0.8579118206059636
0.8724982333195289
0.8239284078034668
0.8574896836961496
0.8809053323763729
0.8399730788112808
0.8575827943975488
0.8932590362179469
0.8705569209655807
0.8356510699417569
0.8353790360216954
0.8535642585871139
0.8442257654538707
0.8853924703961852
0.8556738197141693
0.8788722148033051
0.8672679384665891
0.9002985664991445
0.880581284802885
0.8649823350694855
0.871093048192112
0.877123231130439
0.8850667726148163
0.8759498873559693
0.8858368364159013
0.8626168461736579
0.8694560419996487
0.8684868091552876
0.878832653385483
0.8597057262527906
0.8716029170678088
0.8911292531170336
0.8836632460431576
0.877425791873427
0.8772787595001414
0.8767742118612528
0.8878232052039265
0.8741825274168665
0.8577540714686702
0.8796792749181023
0.8715277768972738
0.9205838916262242
0.8648857903898767
0.8741373896651587
0.8517841715440769
0.8539894631328473
0.8568781263081134
0.8231932955508778
0.8619315661773124

In [10]:
#first, vary c
#for c in range(10, 26):
#    for r in range(10):
#        run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
#        print("c_varying_"+str(c)+"_"+str(r))
#        assert(1==0)

# for alpha in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/alpha_varying_"+str(alpha)+"_"+str(r), alpha=alpha, beta=10, contact_inhib_thresh=10)
#         print("alpha_varying_"+str(alpha)+"_"+str(r))
        
# for h in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/h_varying_"+str(h)+"_"+str(r), beta=10, contact_inhib_thresh=10, h=h)
#         print("h_varying_"+str(h)+"_"+str(r))
        
# for tau in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/tau_varying_"+str(tau)+"_"+str(r), beta=10, contact_inhib_thresh=10, delta=tau)
#         print("tau_varying_"+str(tau)+"_"+str(r))
        
# for sigma in [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/sigma_varying_"+str(sigma)+"_"+str(r), alpha=10, beta=0, contact_inhib_thresh=10, senesc_rate=sigma)
#         print("sigma_varying_"+str(sigma)+"_"+str(r))
        
        

In [8]:
alpha_curv = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, alpha in enumerate([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
                try:
                    curv = measure_curv("alpha_varying/alpha_varying_"+str(alpha)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    alpha_curv[c_][r_][e_][a_] = curv
                    print(alpha, r, elev, azim)
                except:
                    name = "alpha_varying/alpha_varying_"+str(alpha)+"_"+str(r)
                    occupations = np.load("alpha_varying/alpha_varying_"+str(alpha)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("alpha_varying/alpha_varying_"+str(alpha)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    alpha_curv[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.path.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_alpha.npy", alpha_curv)

0.6309777743869786
0.6618928692324217
0.7105681122231171
0.6281923653934293
0.6191055429339941
0.6668102807225644
0.6115373542601783
0.543043236797863
0.6290092416617528
0.627684816117834
0.6278217123718928
0.6580613581220448
0.565604257228946
0.5517294138886812
0.6554857834548982
0.5486205429244594
0.5559497029170967
0.5196948758917329
0.571810285989887
0.5925192401932133
0.6246803612129871
0.5380217297455387
0.7212622653773714
0.6288614107593604
0.5543204086144162
0.5362211608663267
0.5984668691596428
0.5292413001181507
0.5406050453056283
0.6173313695246869
0.6101582843105856
0.582465382903261
0.655856532563101
0.6605773570541182
0.5565925297875892
0.5077159902725675
0.5413283769924292
0.5734260088166658
0.616625245466906
0.6134407727358211
0.5419993764779271
0.6230937932883288
0.5689895804725101
0.5420793000412031
0.5835288196462797
0.5196209204347514
0.539280347116102
0.58935668463855
0.5567805086892124
0.5122134056068833
0.5048967210143498
0.673506362871855
0.6223437729921003
0.57

0.5977390480934375
0.6082641546560895
0.5429607432935619
0.5764105823406231
0.6091632112510551
0.6097544060747193
0.6268174799338933
0.6046857773009081
0.6881243428554153
0.5776576423504375
0.5401074752332243
0.6845099521541521
0.6917044639657484
0.6881721629618052
0.6921329793155786
0.6984993015315425
0.6558143768977361
0.6472084979693885
0.68937551122188
0.6439653391624092
Repeat 9 completed.
0.6822775857846707
0.7069606572646894
0.6799004143894877
0.6737851312390135
0.6949294779944153
0.6561527650055244
0.6447060112810923
0.6388986197771789
0.7167053251616236
0.7525270822648241
0.7155774783875767
0.7006264891291151
0.618381292453711
0.6153594610665855
0.6302512308440039
0.7228037306265228
0.6598608158006206
0.6587506344313748
0.6770756980197973
0.6554496291115253
0.6836073582292793
0.7500166378643275
0.6992430274932361
0.7027186303826959
0.759555109388067
0.7600329173415553
0.6781326649739825
0.63428409091588
0.7201460859257103
0.7730350009837922
0.8376136071673264
0.755644981283619

0.7175785123973485
0.7932883803988968
0.7537173290421963
0.7739840012005456
0.7982589691536145
0.7720212348647372
0.7719329029720425
0.6910986492827376
0.8164712903709705
0.7783669602218168
0.7965191542596299
0.7757947226062667
0.8020626956952769
0.7604629090764107
0.7726011390944902
0.7967574458269548
0.779217137970914
0.7522710502688662
0.7874848832811094
0.734076363721281
0.7698082469138151
0.823900581317994
0.8099382063032158
0.7716856486316757
0.7672283508584927
0.7534339744802145
0.8114617227722883
0.7925945541628936
0.7839994360361137
0.8606666445097225
0.8000908472854906
0.7367296666068418
0.8450745693272463
0.835771846137808
0.8237929718125415
0.7778534853758032
0.7945206856514168
0.7767832570506134
0.794779337583193
0.7776416084561036
0.712582970917068
Repeat 9 completed.


In [9]:
h_curv = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, h in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
                try:
                    curv = measure_curv("h_varying/h_varying_"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    h_curv[c_][r_][e_][a_] = curv
                    print(h, r, elev, azim)
                except:
                    name = "h_varying/h_varying_"+str(h)+"_"+str(r)
                    occupations = np.load("h_varying/h_varying_"+str(h)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("h_varying/h_varying_"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    h_curv[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.path.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_h.npy", h_curv)

0.8757327271552576
0.8547551547753627
0.9000469943873033
0.8339802573541738
0.8746087101055431
0.8825741599719752
0.8386927651487384
0.8556262581679751
0.8832561538191074
0.8897928256074261
0.8778110817840618
0.8744609813212959
0.8628916324005562
0.8637002626273119
0.8554026408026003
0.8619028788033667
0.8824757491850378
0.8574173201932107
0.8847697767237406
0.8702884922032905
0.8977660892284817
0.8465744067685942
0.9005122300129252
0.8843879976309712
0.8802044791041106
0.8857238724477335
0.8766310624747098
0.864910173094699
0.8698874586017067
0.8803442250634468
0.8438443165057902
0.8683297364189048
0.8621684649510648
0.872705738822603
0.8944423136276624
0.8673195860311697
0.8786134818915233
0.8872537953130872
0.8623715116447844
0.873035176605014
0.8761065456925531
0.8479280024092012
0.8825253262661098
0.9020007522654263
0.8480094217740966
0.8899856732123864
0.8769636496167754
0.887747483897965
0.8654993944583181
0.8698442866651183
0.8663850905360594
0.8550505357208978
0.88622011899405

0.7827840563519263
0.741577917968316
0.7383232086694977
0.7721067330073605
0.7452209265460044
0.7557472866326465
0.7564014081875885
0.703756388181596
0.7597520083043391
0.7331232780510122
0.7093703218054781
0.8026926840488452
0.7884490861798238
0.7907894258078778
0.7760490176241149
0.783764794456983
0.7614169366067656
0.8018684462413569
0.7927723734126967
0.7719835167395401
Repeat 9 completed.
0.7314870765447967
0.760303500161141
0.7501715962650166
0.7307388286876041
0.7343264635057979
0.7603033928290955
0.7345085955449576
0.728338857389499
0.7002653022002936
0.705690929883818
0.7079412772043564
0.6971696920207109
0.7206849153766688
0.7099566317695047
0.6406475888615081
0.6480365245734323
0.6413429516512134
0.681276839162935
0.720118020072956
0.7687607506537673
0.7429232781209134
0.7509649669604168
0.7521950057964364
0.7345935871802646
0.7618585777559466
0.7086270102533992
0.7069957843581484
0.6663485142463722
0.6587004043678841
0.7035939214009849
0.7147130328337672
0.6402720272683048


0.5644870066602686
0.5934294518921989
0.5841653602489824
0.6066035535294185
0.5734432247593981
0.5199135589538783
0.5833391130969495
0.6387984507598157
0.6475727945802812
0.6286889930544988
0.5778386441761754
0.4717669751781565
0.44904879604303244
0.4795077594326343
0.5119814880472779
0.571468871098234
0.5563164458816381
0.5083459402136195
0.5404626134290249
0.48077616876963264
0.5250625350436108
0.6187956657272063
0.6032379355965974
0.5554656917955091
0.5633235823876164
0.6007229114059786
0.5270105676441329
0.5399406907852718
0.6968040463660556
0.6286374109017765
0.5317756552540012
0.6730027990016971
0.5590885998764121
0.6214013357842301
0.5798840534287844
0.5549934208292788
0.6234881585437091
0.5560442414078466
0.5378137173424459
0.5420916692099108
0.5303789057051269
Repeat 9 completed.


In [10]:
tau_curv = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, tau in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_intauib_ttaurestau=c)
                try:
                    curv = measure_curv("tau_varying/tau_varying_"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    tau_curv[c_][r_][e_][a_] = curv
                    print(tau, r, elev, azim)
                except:
                    name = "tau_varying/tau_varying_"+str(tau)+"_"+str(r)
                    occupations = np.load("tau_varying/tau_varying_"+str(tau)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("tau_varying/tau_varying_"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    tau_curv[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.pattau.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_tau.npy", tau_curv)

0.3821992496319146
0.3280201926527049
0.35108728694777774
0.46016551192486144
0.4824962437216532
0.47759549076003555
0.4857787916940546
0.5785625089566377
0.47137587127569336
0.3598976381744808
0.4225891791776686
0.36301438280883636
0.36840081804984925
0.47944991562406436
0.5064163580948804
0.5726539071487913
0.5145402032897887
0.4642708872838561
0.41322300890644076
0.4338382439077971
0.3930974316811041
0.5303733185821701
0.491650420948536
0.4251544333691263
0.4649745722625875
0.43637532896840975
0.44023243064044465
0.4612763088779156
0.5098714385865232
0.40180775191543483
0.42648122092312263
0.5016861272313672
0.36778036101025696
0.45363686771975414
0.4842203256096992
0.4259802953919359
0.4212287190166658
0.4495034489541458
0.4025214209346315
0.44441722109720533
0.4739836636256013
0.42422685515748804
0.4521628682999521
0.6007748644795867
0.4224830997745567
0.44205909236637003
0.38413288782633315
0.3238179234924952
0.4125847945181577
0.41082857996401007
0.40534119934375223
0.4486809131

0.47038917101935035
0.4100231254394417
0.4796491192940155
0.5166536609542087
0.4047389682201296
0.38285345019497513
0.42997972961025455
0.40760492480639415
0.4660399265028376
0.5759101901546642
0.4560586763793337
0.48607741312835595
0.4195060736545193
0.4136090843891151
0.3943356508410034
0.48713253077782587
0.42596766899426486
0.4872830988198385
0.47652013965035017
0.4002090947301936
0.3389305073480447
0.3307358470683098
0.43148388906556795
0.4856859764175354
0.42179646850666014
0.40619238034257016
0.4654322431776105
0.5048941665733467
0.48350493016444807
Repeat 9 completed.
0.5383556907977893
0.545762855529167
0.46159094375057863
0.5226780570865832
0.5123427323727547
0.432606918017733
0.4418866219068819
0.4757796376566215
0.5033683623365179
0.3657830437335513
0.5059746721838072
0.4835612527342615
0.46265316054360095
0.4094343147650598
0.42214637569861085
0.5940174775781033
0.5264122447672537
0.47114225496707113
0.5260122883663579
0.5392900793699436
0.45825015897785026
0.4922744126273

0.5547984273164868
0.5516246685001744
0.5439173324432146
0.5412550777976078
0.5968883362176371
0.5420710459332505
0.5824205904546754
0.6126774237425576
0.6774722154198455
0.6026311367555809
0.49534449416338194
0.5919465350250284
0.657762030336802
0.5496780439679383
0.50069448061569
0.47947571161310953
0.5736463501207901
0.5203034453924584
0.7117736245881349
0.6498444854153062
0.6130537712481021
0.6177295559438534
0.571162209597929
0.6557387821321723
0.5949608009709751
0.5888234780733522
0.5578283088878905
0.556573748019993
0.6187741913651891
0.5878436260808665
0.579437283037604
0.5186379780454606
0.59056112218544
0.5892932290701908
0.6181586837682701
0.5795745247814637
0.677158521657287
0.5472047131518956
0.4697036675132489
0.5834285679514548
0.5939014605819375
0.5815173627573398
0.44046469784458697
0.4925394223834459
0.659946780832255
0.46745060668572513
0.48374309557746364
0.6428707611146602
0.6339413833211434
0.5704423219217977
0.5710740732010933
0.6653664326959333
0.666167653081893

In [11]:
sigma_curv = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, sigma in enumerate([0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_insigmaib_tsigmaressigma=c)
                try:
                    curv = measure_curv("sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    sigma_curv[c_][r_][e_][a_] = curv
                    print(sigma, r, elev, azim)
                except:
                    name = "sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)
                    occupations = np.load("sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    sigma_curv[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.patsigma.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_sigma.npy", sigma_curv)

0.9059263271197407
0.9077960994199773
0.8901653582225914
0.8944677294967533
0.8704955177222735
0.8816079077448291
0.84670225706984
0.8854886501742103
0.875289620094219
0.8834900010291791
0.888931485989292
0.9028276927639487
0.8978768997233768
0.8867891111948096
0.8629928994610492
0.8749629016457704
0.8600742761291303
0.8760860816563074
0.8706925256464739
0.8950095881803141
0.9001281391306543
0.8685015696800115
0.8874319486749694
0.8931637431554416
0.8663757738397698
0.8850749911862139
0.8958873970382896
0.8708402122449529
0.8823463276759149
0.869104362053366
0.8793468445182302
0.9103192186507371
0.8617063405448762
0.8841211672155233
0.873876391233243
0.8876353015806636
0.8977313607377109
0.8829725841875109
0.8706305256890428
0.8893928464366834
0.8807772498031802
0.8809539607938379
0.8892227892425053
0.863424166378154
0.8976926676183892
0.9155078970766032
0.8934215535337615
0.8979939467776027
0.8758092831745296
0.8633063708541647
0.875290762146898
0.8684096731124408
0.8907156746060372
0

0.1259725242166208
0.1259725242166208
0.8928318318561831
0.8679029207020554
0.8627260951573222
0.8347573187741757
0.8739134811884677
0.8712663106545754
0.8055849469926978
0.8561573233150034
0.8873233068881977
0.8940453391508455
0.8695658676862112
0.8921866596717215
0.8668353675813725
0.8614336683346346
0.8718214494810718
0.849279264139098
0.8853778633435195
0.8958690187574172
Repeat 9 completed.
0.9037199273648435
0.8720269108131941
0.8642583904197862
0.8502650733085709
0.8664982168076937
0.8836905795445006
0.8547014273514908
0.8740120651222864
0.9120523624233055
0.9024936401910485
0.8807440363453807
0.8973748867549176
0.8889625858359803
0.8673497416649867
0.8698026771405883
0.9016500555674056
0.8698912744124847
0.8894183952224934
0.9112496111388584
0.8813034710296049
0.9041286440411593
0.8543257000365454
0.8790559197874962
0.8884259314498418
0.8550959615444905
0.8779722039297827
0.8856554614033865
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242

0.88115011452919
0.8567285402492393
0.8642775254505664
0.8745413691649011
0.8802479824450639
0.04937621339747351
0.04937621339747351
0.04937621339747351
0.023128822764171227
0.023128822764171227
0.023128822764171227
0.023128822764171216
0.023128822764171216
0.023128822764171216
0.8945942675088415
0.8801484581533809
0.9112959697511527
0.8446595184468354
0.8804795930137238
0.893199113957901
0.8677747256717073
0.8853737380062844
0.8657465322205979
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.1259725242166208
0.8728940383171863
0.8707088053960498
0.8980840233309931
0.8918830236421322
0.8628876301672498
0.8631177550534331
0.852751550272384
0.8713884039325894
0.8930114848441049
Repeat 9 completed.


In [26]:
# #now generate figures for fig3- this time only the necessary ones
# for beta in range(1, 11):
#     for r in range(10):
#         run_plot_save("beta_varying_fig3/"+str(beta)+"_"+str(r), alpha=1, beta=beta, contact_inhib_thresh=20, h=1, delta=1, max_cells=10000)
#         print("beta_varying_fig3_"+str(beta)+"_"+str(r))

# for h in list(np.power(10, np.arange(-5, 5))):
#     for r in range(10):
#         run_plot_save("h_varying_fig3/"+str(h)+"_"+str(r), alpha=1, beta=2, contact_inhib_thresh=20, h=h, delta=1, max_cells=10000)
#         print("h_varying_fig3_"+str(h)+"_"+str(r))


# for tau in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for r in range(10):
#         run_plot_save("tau_varying_fig3/"+str(tau)+"_"+str(r), alpha=1, beta=2, contact_inhib_thresh=20, h=1, delta=tau, max_cells=10000)
#         print("tau_varying_fig3_"+str(tau)+"_"+str(r))


# for c in range(10, 26):
#     for r in range(10):
#         run_plot_save("c_varying_fig3/"+str(c)+"_"+str(r), alpha=1, beta=2, contact_inhib_thresh=c, h=1, delta=1, max_cells=10000)
#         print("c_varying_fig3_"+str(c)+"_"+str(r))


KeyboardInterrupt: 

In [17]:
beta_curv_fig3 = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, beta in enumerate(range(1, 11)):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inbetaib_tbetaresbeta=c)
                try:
                    curv = measure_curv("beta_varying_fig3/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    beta_curv_fig3[c_][r_][e_][a_] = curv
                    print(beta, r, elev, azim)
                except:
                    name = "beta_varying_fig3/"+str(beta)+"_"+str(r)
                    occupations = np.load("beta_varying_fig3/"+str(beta)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("beta_varying_fig3/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    beta_curv_fig3[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.patbeta.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_beta_fig3.npy", beta_curv_fig3)

0.9334031954737843
1 0 -60 -60
0.9287483123597333
1 0 -60 -20
0.9389515451813334
1 0 -60 20
0.9056207510094808
1 0 -20 -60
0.9210841924436304
1 0 -20 -20
0.910240974026833
1 0 -20 20
0.9039208615992477
1 0 20 -60
0.9203579465869601
1 0 20 -20
0.9142531935336129
1 0 20 20
0.9293425672508833
1 1 -60 -60
0.9087838482310826
1 1 -60 -20
0.9087892525603982
1 1 -60 20
0.9130420714487734
1 1 -20 -60
0.9063967514818868
1 1 -20 -20
0.8871071075532246
1 1 -20 20
0.916028992785155
1 1 20 -60
0.9113001248879703
1 1 20 -20
0.8969337936682933
1 1 20 20
0.949936704105868
1 2 -60 -60
0.93037523121895
1 2 -60 -20
0.9243010967606801
1 2 -60 20
0.9071798041554334
1 2 -20 -60
0.9176662737036616
1 2 -20 -20
0.9218879669175927
1 2 -20 20
0.9147152953587471
1 2 20 -60
0.9036621475078652
1 2 20 -20
0.9223573559701834
1 2 20 20
0.9405446937356124
1 3 -60 -60
0.9429419654868612
1 3 -60 -20
0.9512118161751653
1 3 -60 20
0.9020207986982307
1 3 -20 -60
0.9211322102402377
1 3 -20 -20
0.8962328680311825
1 3 -20 20
0.

0.9451534234736466
4 0 -60 -60
0.9446496588901212
4 0 -60 -20
0.9128183893476598
4 0 -60 20
0.8959393525441801
4 0 -20 -60
0.9080751717400612
4 0 -20 -20
0.905631551731009
4 0 -20 20
0.8971168372309389
4 0 20 -60
0.915770209153227
4 0 20 -20
0.9179405112681419
4 0 20 20
0.9394191808843669
4 1 -60 -60
0.933454023840993
4 1 -60 -20
0.934538670613363
4 1 -60 20
0.9060744058760581
4 1 -20 -60
0.8958709155667134
4 1 -20 -20
0.9153349206940022
4 1 -20 20
0.9221890566963307
4 1 20 -60
0.909586614462045
4 1 20 -20
0.9120920214431305
4 1 20 20
0.943290714125316
4 2 -60 -60
0.9378715943417401
4 2 -60 -20
0.937290516920552
4 2 -60 20
0.919135599178425
4 2 -20 -60
0.8860333230002196
4 2 -20 -20
0.9151154820835476
4 2 -20 20
0.9170624770215404
4 2 20 -60
0.9119962513709612
4 2 20 -20
0.912917015935094
4 2 20 20
0.938452857331885
4 3 -60 -60
0.9120470405577701
4 3 -60 -20
0.9284948398119234
4 3 -60 20
0.8931340746434645
4 3 -20 -60
0.9113330512871295
4 3 -20 -20
0.8902286586377213
4 3 -20 20
0.90167

0.9282330330837424
7 0 -60 -20
0.9105161095778371
7 0 -60 20
0.8891392897492297
7 0 -20 -60
0.9109199157123326
7 0 -20 -20
0.9071072455945692
7 0 -20 20
0.8891168120538913
7 0 20 -60
0.8846474891322599
7 0 20 -20
0.8964620227806555
7 0 20 20
0.9219680679485726
7 1 -60 -60
0.913538755120051
7 1 -60 -20
0.9230357342936171
7 1 -60 20
0.8976838278607825
7 1 -20 -60
0.9092878635640361
7 1 -20 -20
0.9118169480192525
7 1 -20 20
0.9100758774136933
7 1 20 -60
0.9084581175729273
7 1 20 -20
0.9006899503379096
7 1 20 20
0.9248198372740283
7 2 -60 -60
0.9132351910014606
7 2 -60 -20
0.928316370248174
7 2 -60 20
0.9326690601416032
7 2 -20 -60
0.9163039650914985
7 2 -20 -20
0.9040876609903932
7 2 -20 20
0.8934425979457187
7 2 20 -60
0.9081599357375622
7 2 20 -20
0.9173231814315133
7 2 20 20
0.940581658673663
7 3 -60 -60
0.9257446984406676
7 3 -60 -20
0.9156997172773231
7 3 -60 20
0.8896483318827364
7 3 -20 -60
0.8748227284086227
7 3 -20 -20
0.8945681233257747
7 3 -20 20
0.8975011296498541
7 3 20 -60
0

0.926834286515164
10 0 -60 -20
0.9214175152839698
10 0 -60 20
0.9211803791807786
10 0 -20 -60
0.885025506581488
10 0 -20 -20
0.9016676922237291
10 0 -20 20
0.8989727883004885
10 0 20 -60
0.9021588454521299
10 0 20 -20
0.8858818422849253
10 0 20 20
0.9082255507533318
10 1 -60 -60
0.9379828950836907
10 1 -60 -20
0.9281925167583512
10 1 -60 20
0.9118474709204534
10 1 -20 -60
0.9125629319923182
10 1 -20 -20
0.910256094242655
10 1 -20 20
0.9093803203543419
10 1 20 -60
0.9037954315459393
10 1 20 -20
0.8998423664718085
10 1 20 20
0.9231578125398134
10 2 -60 -60
0.9255981876676156
10 2 -60 -20
0.9096801922194752
10 2 -60 20
0.8917255722031793
10 2 -20 -60
0.9104885837496216
10 2 -20 -20
0.9025979627083691
10 2 -20 20
0.8950766439432161
10 2 20 -60
0.9178399869498877
10 2 20 -20
0.9165190197873025
10 2 20 20
0.942889250497826
10 3 -60 -60
0.9311951296943659
10 3 -60 -20
0.9161037938258714
10 3 -60 20
0.9093762956198022
10 3 -20 -60
0.889165645034029
10 3 -20 -20
0.8823010230025011
10 3 -20 20
0

In [18]:
h_curv_fig3 = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, h in enumerate([0.01, 0.1, 1.0, 10, 100]):
    for r_, r in enumerate(range(10)):
         for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                try:
                    curv = measure_curv("h_varying_fig3/"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    h_curv_fig3[c_][r_][e_][a_] = curv
                    print(h, r, elev, azim)
                except:
                    name = "h_varying_fig3/"+str(h)+"_"+str(r)
                    occupations = np.load("h_varying_fig3/"+str(h)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("h_varying_fig3/"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    h_curv_fig3[c_][r_][e_][a_] = curv
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_h_fig3.npy", h_curv_fig3)

0.9350545100549627
0.01 0 -60 -60
0.9315507019240796
0.01 0 -60 -20
0.9307444504815304
0.01 0 -60 20
0.9016614391244168
0.01 0 -20 -60
0.9110665286031848
0.01 0 -20 -20
0.9283255564336925
0.01 0 -20 20
0.9222005052487721
0.01 0 20 -60
0.9154580683549188
0.01 0 20 -20
0.9101784896585464
0.01 0 20 20
0.937293033800821
0.01 1 -60 -60
0.9336656469318915
0.01 1 -60 -20
0.9349681523697974
0.01 1 -60 20
0.9065582713742759
0.01 1 -20 -60
0.9173378838458144
0.01 1 -20 -20
0.8961798710684377
0.01 1 -20 20
0.9091145456432168
0.01 1 20 -60
0.9382165364728324
0.01 1 20 -20
0.9222954418472906
0.01 1 20 20
0.940809081433357
0.01 2 -60 -60
0.9255035951902724
0.01 2 -60 -20
0.9261887740608582
0.01 2 -60 20
0.9268635394580209
0.01 2 -20 -60
0.9079692066351943
0.01 2 -20 -20
0.9022931646170286
0.01 2 -20 20
0.9110105458232667
0.01 2 20 -60
0.8926957108046467
0.01 2 20 -20
0.9271077343334895
0.01 2 20 20
0.9208928124053386
0.01 3 -60 -60
0.921925181880519
0.01 3 -60 -20
0.9232809784257237
0.01 3 -60 20
0.

0.9302966453983231
1.0 8 -60 -60
0.9224742814038933
1.0 8 -60 -20
0.9355899447727863
1.0 8 -60 20
0.9169246616150716
1.0 8 -20 -60
0.9007892524359468
1.0 8 -20 -20
0.9164725176552073
1.0 8 -20 20
0.9206246389904277
1.0 8 20 -60
0.9233159440917699
1.0 8 20 -20
0.9095883353741102
1.0 8 20 20
0.9344865689895593
1.0 9 -60 -60
0.9316812075471036
1.0 9 -60 -20
0.9359809991232588
1.0 9 -60 20
0.9140399151599594
1.0 9 -20 -60
0.9148306621202095
1.0 9 -20 -20
0.8838272950316854
1.0 9 -20 20
0.9032911573510702
1.0 9 20 -60
0.8963107588749719
1.0 9 20 -20
0.8888938246687771
1.0 9 20 20
Repeat 9 completed.
0.9274922271153125
10 0 -60 -60
0.9164894875474822
10 0 -60 -20
0.9454831940516495
10 0 -60 20
0.8946407870149765
10 0 -20 -60
0.906151317760833
10 0 -20 -20
0.8852471490733415
10 0 -20 20
0.8809120589335652
10 0 20 -60
0.9156931670589614
10 0 20 -20
0.9131412116124952
10 0 20 20
0.9272921369860083
10 1 -60 -60
0.9351088055997738
10 1 -60 -20
0.941832335734333
10 1 -60 20
0.9030922443553638
10 1

In [15]:
tau_curv_fig3 = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, tau in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), tau=10, contact_intauib_ttaurestau=c)
                try:
                    curv = measure_curv("tau_varying_fig3/"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    tau_curv_fig3[c_][r_][e_][a_] = curv
                    print(tau, r, elev, azim)
                except:
                    name = "tau_varying_fig3/"+str(tau)+"_"+str(r)
                    occupations = np.load("tau_varying_fig3/"+str(tau)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("tau_varying_fig3/"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    tau_curv_fig3[c_][r_][e_][a_] = curv
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_tau_fig3.npy", tau_curv_fig3)

0.7917610013959098
0.1 0 -60 -60
0.7051284155967957
0.1 0 -60 -20
0.8229883845587986
0.1 0 -60 20
0.7140072484794793
0.1 0 -20 -60
0.7558636643737905
0.1 0 -20 -20
0.7849950885146062
0.1 0 -20 20
0.7189588804846081
0.1 0 20 -60
0.7565871811986087
0.1 0 20 -20
0.7530184495743423
0.1 0 20 20
0.7977246387000084
0.1 1 -60 -60
0.8513576545863076
0.1 1 -60 -20
0.8226653498478332
0.1 1 -60 20
0.7857587853114157
0.1 1 -20 -60
0.7539279761118827
0.1 1 -20 -20
0.729502255178156
0.1 1 -20 20
0.7428858609618822
0.1 1 20 -60
0.7974117151386193
0.1 1 20 -20
0.8030906254940461
0.1 1 20 20
0.7487132938481905
0.1 2 -60 -60
0.7515045615414969
0.1 2 -60 -20
0.7897213350102691
0.1 2 -60 20
0.7271574987893681
0.1 2 -20 -60
0.7979277724408829
0.1 2 -20 -20
0.7948639335825418
0.1 2 -20 20
0.8031982481028133
0.1 2 20 -60
0.7507725137259358
0.1 2 20 -20
0.7379029251167162
0.1 2 20 20
0.8491128354042694
0.1 3 -60 -60
0.8173264445493662
0.1 3 -60 -20
0.8040038698550478
0.1 3 -60 20
0.7410695501326134
0.1 3 -20 -

0.7984431752004909
0.3 8 -60 20
0.7642998139244994
0.3 8 -20 -60
0.7874770576389015
0.3 8 -20 -20
0.8496250957594756
0.3 8 -20 20
0.7691151108065478
0.3 8 20 -60
0.8250679016552268
0.3 8 20 -20
0.786800878686583
0.3 8 20 20
0.8401562417881475
0.3 9 -60 -60
0.8326438690221406
0.3 9 -60 -20
0.8552808255890287
0.3 9 -60 20
0.8816100479591488
0.3 9 -20 -60
0.8250608156881537
0.3 9 -20 -20
0.8567797762722605
0.3 9 -20 20
0.8231303665919353
0.3 9 20 -60
0.8638831387672451
0.3 9 20 -20
0.7885082671633349
0.3 9 20 20
Repeat 9 completed.
0.8300133037432247
0.4 0 -60 -60
0.8374462100962247
0.4 0 -60 -20
0.8420718557256833
0.4 0 -60 20
0.8873286412667319
0.4 0 -20 -60
0.8406101255963182
0.4 0 -20 -20
0.8687577732432813
0.4 0 -20 20
0.8574923929939561
0.4 0 20 -60
0.8607270686934198
0.4 0 20 -20
0.8496238129051826
0.4 0 20 20
0.8559432169713065
0.4 1 -60 -60
0.8688511695576392
0.4 1 -60 -20
0.8810395028795738
0.4 1 -60 20
0.8652970819950413
0.4 1 -20 -60
0.8807201375670289
0.4 1 -20 -20
0.85651004

0.9034646779699548
0.6 6 -20 -20
0.8880541855833828
0.6 6 -20 20
0.8868086250828146
0.6 6 20 -60
0.9119710166474225
0.6 6 20 -20
0.9025543285340737
0.6 6 20 20
0.9191801427004889
0.6 7 -60 -60
0.9004304494708137
0.6 7 -60 -20
0.9209914948575574
0.6 7 -60 20
0.9018945554285993
0.6 7 -20 -60
0.8933381824661882
0.6 7 -20 -20
0.8892907702446156
0.6 7 -20 20
0.9101539988803963
0.6 7 20 -60
0.8982512877291133
0.6 7 20 -20
0.9097845012345097
0.6 7 20 20
0.8999243325136562
0.6 8 -60 -60
0.9068716138783312
0.6 8 -60 -20
0.9099079278400555
0.6 8 -60 20
0.8600181439881455
0.6 8 -20 -60
0.8590653184296881
0.6 8 -20 -20
0.8865093223819306
0.6 8 -20 20
0.8698161302318341
0.6 8 20 -60
0.8700347257559079
0.6 8 20 -20
0.888816947477602
0.6 8 20 20
0.9062505167144348
0.6 9 -60 -60
0.9125125683622316
0.6 9 -60 -20
0.8958672459888125
0.6 9 -60 20
0.8558271474690747
0.6 9 -20 -60
0.8719873761841465
0.6 9 -20 -20
0.8593479136954093
0.6 9 -20 20
0.8849641387592608
0.6 9 20 -60
0.8636443467713559
0.6 9 20 -20

0.8968349502368557
0.9 4 20 -60
0.8860625815961382
0.9 4 20 -20
0.9035874624516333
0.9 4 20 20
0.9225026650323107
0.9 5 -60 -60
0.9138622009307736
0.9 5 -60 -20
0.9096012704024271
0.9 5 -60 20
0.9162616760862389
0.9 5 -20 -60
0.9024087182485065
0.9 5 -20 -20
0.915254585255525
0.9 5 -20 20
0.891963116263865
0.9 5 20 -60
0.9002092140062713
0.9 5 20 -20
0.9136721421040326
0.9 5 20 20
0.9040493015382904
0.9 6 -60 -60
0.9136991018753217
0.9 6 -60 -20
0.9306612417939658
0.9 6 -60 20
0.9063479281326168
0.9 6 -20 -60
0.8741772445444997
0.9 6 -20 -20
0.8836425191590507
0.9 6 -20 20
0.8765230237595768
0.9 6 20 -60
0.8924105374329386
0.9 6 20 -20
0.887895770409772
0.9 6 20 20
0.9286131105636083
0.9 7 -60 -60
0.9371053719004346
0.9 7 -60 -20
0.9266253293283151
0.9 7 -60 20
0.934913251387905
0.9 7 -20 -60
0.9097441579758172
0.9 7 -20 -20
0.894164548039801
0.9 7 -20 20
0.9173788966849011
0.9 7 20 -60
0.8934372590275227
0.9 7 20 -20
0.8973025158285804
0.9 7 20 20
0.942355879483351
0.9 8 -60 -60
0.923

In [20]:
c_curv_fig3 = np.zeros((16, 10, 3, 3))



#first, vary c
for c_, c in enumerate(range(10, 26)):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), c=10, contact_incib_tcresc=c)
                try:
                    curv = measure_curv("c_varying_fig3/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    c_curv_fig3[c_][r_][e_][a_] = curv
                    print(c, r, elev, azim)
                except:
                    name = "c_varying_fig3/"+str(c)+"_"+str(r)
                    occupations = np.load("c_varying_fig3/"+str(c)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("c_varying_fig3/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    c_curv_fig3[c_][r_][e_][a_] = curv
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_c_fig3.npy", c_curv_fig3)

0.9608825721314536
10 0 -60 -60
0.9277507415799493
10 0 -60 -20
0.9364333074208515
10 0 -60 20
0.8945820688852405
10 0 -20 -60
0.9119823041280828
10 0 -20 -20
0.8965922643956731
10 0 -20 20
0.8999676292549993
10 0 20 -60
0.9002763903562376
10 0 20 -20
0.8955587915966435
10 0 20 20
0.9290042577104937
10 1 -60 -60
0.9351715112648635
10 1 -60 -20
0.933890633721875
10 1 -60 20
0.8928876034692015
10 1 -20 -60
0.9014719513580904
10 1 -20 -20
0.898126626475435
10 1 -20 20
0.9265246929393144
10 1 20 -60
0.9055478491036885
10 1 20 -20
0.9155258925307875
10 1 20 20
0.9374940170877828
10 2 -60 -60
0.9298148168556287
10 2 -60 -20
0.9186728747284071
10 2 -60 20
0.907785281250726
10 2 -20 -60
0.9077862402062207
10 2 -20 -20
0.9120303809176915
10 2 -20 20
0.9104651210947514
10 2 20 -60
0.9245730081508625
10 2 20 -20
0.9237385928675692
10 2 20 20
0.9365024533127819
10 3 -60 -60
0.9269830294753362
10 3 -60 -20
0.9279002237750488
10 3 -60 20
0.9015242444293553
10 3 -20 -60
0.9042901838972176
10 3 -20 -2

0.9129170242405261
12 9 -60 -20
0.9247986123390186
12 9 -60 20
0.8918223492426903
12 9 -20 -60
0.9177728890960102
12 9 -20 -20
0.919386162197493
12 9 -20 20
0.8945573151359758
12 9 20 -60
0.9190290188173786
12 9 20 -20
0.9130773425521435
12 9 20 20
Repeat 9 completed.
0.9352798963496696
13 0 -60 -60
0.9237870265323643
13 0 -60 -20
0.9287771283106345
13 0 -60 20
0.9113918305029749
13 0 -20 -60
0.8963529836001337
13 0 -20 -20
0.9079263497942437
13 0 -20 20
0.9063368798438919
13 0 20 -60
0.9208713262090843
13 0 20 -20
0.9169954207074882
13 0 20 20
0.9343038789441366
13 1 -60 -60
0.9173379333820966
13 1 -60 -20
0.9228403103545766
13 1 -60 20
0.9123090701379007
13 1 -20 -60
0.9075343059279513
13 1 -20 -20
0.9233740928966692
13 1 -20 20
0.9128627289462944
13 1 20 -60
0.9144676222346649
13 1 20 -20
0.9207054270404353
13 1 20 20
0.9397331940938054
13 2 -60 -60
0.9493851573253671
13 2 -60 -20
0.9476052852578939
13 2 -60 20
0.9233124430898212
13 2 -20 -60
0.9157482451092334
13 2 -20 -20
0.941087

0.9350838344473119
15 8 -60 -20
0.926008948798784
15 8 -60 20
0.9255959251806659
15 8 -20 -60
0.9074127694271815
15 8 -20 -20
0.8882075592173494
15 8 -20 20
0.9267936819291169
15 8 20 -60
0.9176330448921818
15 8 20 -20
0.8982191292112217
15 8 20 20
0.9286884883799589
15 9 -60 -60
0.9255623893898294
15 9 -60 -20
0.9366345321216368
15 9 -60 20
0.9108178717066069
15 9 -20 -60
0.8820924523353498
15 9 -20 -20
0.9082696481549177
15 9 -20 20
0.9297728345242571
15 9 20 -60
0.8959223588635121
15 9 20 -20
0.8946643853430506
15 9 20 20
Repeat 9 completed.
0.9260936724918086
16 0 -60 -60
0.9334209540178927
16 0 -60 -20
0.9297752535004317
16 0 -60 20
0.9053584894866791
16 0 -20 -60
0.8959048010211809
16 0 -20 -20
0.8946790244029711
16 0 -20 20
0.903855503194777
16 0 20 -60
0.9270547654729111
16 0 20 -20
0.9180285378289104
16 0 20 20
0.9260667511951352
16 1 -60 -60
0.9405016126963359
16 1 -60 -20
0.934714566283845
16 1 -60 20
0.9116430541494679
16 1 -20 -60
0.8865317237937754
16 1 -20 -20
0.88864079

0.8923991320002836
18 7 -60 20
0.8954926919812366
18 7 -20 -60
0.9264844093099285
18 7 -20 -20
0.9173869047634756
18 7 -20 20
0.9110517765431704
18 7 20 -60
0.9036241587714149
18 7 20 -20
0.9223604824266574
18 7 20 20
0.9241776920832994
18 8 -60 -60
0.9237911859600643
18 8 -60 -20
0.9306455763539039
18 8 -60 20
0.8991430411358834
18 8 -20 -60
0.9039712167295484
18 8 -20 -20
0.9251121670336387
18 8 -20 20
0.912750600684999
18 8 20 -60
0.9208180950543599
18 8 20 -20
0.9151009774230421
18 8 20 20
0.9261755219342028
18 9 -60 -60
0.9298365187098756
18 9 -60 -20
0.9360146900612756
18 9 -60 20
0.8804815139560256
18 9 -20 -60
0.9218583431939754
18 9 -20 -20
0.9094593879759632
18 9 -20 20
0.9066203358989632
18 9 20 -60
0.9009076592361286
18 9 20 -20
0.9024051499874676
18 9 20 20
Repeat 9 completed.
0.9362586837415036
19 0 -60 -60
0.9400479731392362
19 0 -60 -20
0.9295895101953712
19 0 -60 20
0.9006849426278767
19 0 -20 -60
0.9064169161038881
19 0 -20 -20
0.9017511353014991
19 0 -20 20
0.9050160

0.9147679306062599
21 6 -20 -60
0.9087560364229706
21 6 -20 -20
0.9037816564844834
21 6 -20 20
0.9090082947789432
21 6 20 -60
0.8926309430121636
21 6 20 -20
0.8944660672051765
21 6 20 20
0.9277540284420029
21 7 -60 -60
0.9274932389292901
21 7 -60 -20
0.926912369721656
21 7 -60 20
0.9093115974164564
21 7 -20 -60
0.9122139900306919
21 7 -20 -20
0.9188165093980721
21 7 -20 20
0.9050099016128208
21 7 20 -60
0.9109743700626336
21 7 20 -20
0.9235102198031855
21 7 20 20
0.9132220450248131
21 8 -60 -60
0.9129437425329915
21 8 -60 -20
0.9304835758740275
21 8 -60 20
0.9320131644588172
21 8 -20 -60
0.9040556164510887
21 8 -20 -20
0.9122247655621143
21 8 -20 20
0.9264486860676613
21 8 20 -60
0.9079488686159569
21 8 20 -20
0.9183105580342891
21 8 20 20
0.9191165798921954
21 9 -60 -60
0.9206793266447166
21 9 -60 -20
0.9537018382168096
21 9 -60 20
0.9001544933785006
21 9 -20 -60
0.9124334682134756
21 9 -20 -20
0.9034665571370007
21 9 -20 20
0.9047698857911052
21 9 20 -60
0.9172899145332883
21 9 20 -2

0.9151535006706402
24 5 -20 -20
0.8944959266722728
24 5 -20 20
0.9039071051154828
24 5 20 -60
0.9203397621557673
24 5 20 -20
0.913124032142694
24 5 20 20
0.9226918675512836
24 6 -60 -60
0.9023808078807791
24 6 -60 -20
0.910666860837409
24 6 -60 20
0.896729355248171
24 6 -20 -60
0.9180872080559085
24 6 -20 -20
0.9238082808862433
24 6 -20 20
0.9009047009110506
24 6 20 -60
0.9246007959701863
24 6 20 -20
0.9176422897624928
24 6 20 20
0.9445679461163236
24 7 -60 -60
0.934907349736861
24 7 -60 -20
0.9416100534107767
24 7 -60 20
0.9103727780436471
24 7 -20 -60
0.8933138024029695
24 7 -20 -20
0.9165139775138212
24 7 -20 20
0.8991073268540148
24 7 20 -60
0.918400912941782
24 7 20 -20
0.8996668291935243
24 7 20 20
0.9007587201077875
24 8 -60 -60
0.9140890101141643
24 8 -60 -20
0.9045585289341574
24 8 -60 20
0.9069497270610358
24 8 -20 -60
0.8897603631396257
24 8 -20 -20
0.8999558963853601
24 8 -20 20
0.9003109023706503
24 8 20 -60
0.8891775571884775
24 8 20 -20
0.9058242229971759
24 8 20 20
0.90

In [21]:
alpha_curv_fig4 = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, alpha in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), alpha=10, contact_inalphaib_talpharesalpha=c)
                try:
                    curv = measure_curv("alpha_varying_fig4/"+str(alpha)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    alpha_curv_fig4[c_][r_][e_][a_] = curv
                    print(alpha, r, elev, azim)
                except:
                    name = "alpha_varying_fig4/"+str(alpha)+"_"+str(r)
                    occupations = np.load("alpha_varying_fig4/"+str(alpha)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("alpha_varying_fig4/"+str(alpha)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    alpha_curv_fig4[c_][r_][e_][a_] = curv
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_alpha_fig4.npy", alpha_curv_fig4)

0.9313791605733348
0.1 0 -60 -60
0.9205380669910314
0.1 0 -60 -20
0.9363803341450027
0.1 0 -60 20
0.9026407308955516
0.1 0 -20 -60
0.9178112715996747
0.1 0 -20 -20
0.9052759652975858
0.1 0 -20 20
0.9074866167864296
0.1 0 20 -60
0.9110695036667494
0.1 0 20 -20
0.9169162367836592
0.1 0 20 20
0.9219176716957761
0.1 1 -60 -60
0.9361763707249791
0.1 1 -60 -20
0.9239767225004425
0.1 1 -60 20
0.8764127841410795
0.1 1 -20 -60
0.9099594302650383
0.1 1 -20 -20
0.8980148233589266
0.1 1 -20 20
0.9162567480762827
0.1 1 20 -60
0.9074594924457989
0.1 1 20 -20
0.9086383824624766
0.1 1 20 20
0.9353508592655675
0.1 2 -60 -60
0.9189892925853526
0.1 2 -60 -20
0.9317673541652625
0.1 2 -60 20
0.9115206768710179
0.1 2 -20 -60
0.9022801593742581
0.1 2 -20 -20
0.9106379904440732
0.1 2 -20 20
0.9186803822864478
0.1 2 20 -60
0.9100530217380459
0.1 2 20 -20
0.9110131391087131
0.1 2 20 20
0.9439663501250011
0.1 3 -60 -60
0.937514620197984
0.1 3 -60 -20
0.9329312473421767
0.1 3 -60 20
0.9236047977735291
0.1 3 -20 -

0.9297847802263753
0.3 8 -60 20
0.91229711274761
0.3 8 -20 -60
0.9071073698704345
0.3 8 -20 -20
0.9096254879245282
0.3 8 -20 20
0.9123253221096215
0.3 8 20 -60
0.8912460857365345
0.3 8 20 -20
0.9198405259561949
0.3 8 20 20
0.9500579348343087
0.3 9 -60 -60
0.9388576472083751
0.3 9 -60 -20
0.9221970465994157
0.3 9 -60 20
0.9026329470480792
0.3 9 -20 -60
0.9153830255344183
0.3 9 -20 -20
0.9124208150262169
0.3 9 -20 20
0.8982430330155141
0.3 9 20 -60
0.9155111370104405
0.3 9 20 -20
0.9031984270781119
0.3 9 20 20
Repeat 9 completed.
0.9411943500391917
0.4 0 -60 -60
0.9143288386019498
0.4 0 -60 -20
0.9124039414722463
0.4 0 -60 20
0.9214288757384613
0.4 0 -20 -60
0.9171531718657426
0.4 0 -20 -20
0.8808908303864915
0.4 0 -20 20
0.9098209579926541
0.4 0 20 -60
0.9063763123360828
0.4 0 20 -20
0.9172690915129246
0.4 0 20 20
0.9278056287751784
0.4 1 -60 -60
0.9304126615956693
0.4 1 -60 -20
0.9361986267889494
0.4 1 -60 20
0.916021501471522
0.4 1 -20 -60
0.9204648207490594
0.4 1 -20 -20
0.9303571678

0.9144252238819974
0.6 6 -20 -20
0.9218448573393905
0.6 6 -20 20
0.9089538505779645
0.6 6 20 -60
0.8950847738569689
0.6 6 20 -20
0.9188869096768618
0.6 6 20 20
0.9179684343019605
0.6 7 -60 -60
0.920867391186723
0.6 7 -60 -20
0.9301319688402564
0.6 7 -60 20
0.9051137342007581
0.6 7 -20 -60
0.9292777515297599
0.6 7 -20 -20
0.911298340631275
0.6 7 -20 20
0.9002114253007711
0.6 7 20 -60
0.9064508752607668
0.6 7 20 -20
0.9041092952595078
0.6 7 20 20
0.9339609125746743
0.6 8 -60 -60
0.9245488453112863
0.6 8 -60 -20
0.938803676984812
0.6 8 -60 20
0.9133233944558814
0.6 8 -20 -60
0.8997227720898548
0.6 8 -20 -20
0.9097810092442059
0.6 8 -20 20
0.9204803870949635
0.6 8 20 -60
0.9095457932648379
0.6 8 20 -20
0.9085281677109351
0.6 8 20 20
0.9389653334350794
0.6 9 -60 -60
0.9245105563865255
0.6 9 -60 -20
0.945643934521647
0.6 9 -60 20
0.9214384305939436
0.6 9 -20 -60
0.9092701478490981
0.6 9 -20 -20
0.9395546427040911
0.6 9 -20 20
0.9141459903553781
0.6 9 20 -60
0.9031441963604799
0.6 9 20 -20
0.

0.905318895346276
0.9 4 20 -60
0.8960175190998968
0.9 4 20 -20
0.9098265216174525
0.9 4 20 20
0.9303172102942728
0.9 5 -60 -60
0.925853910403373
0.9 5 -60 -20
0.9248685587429096
0.9 5 -60 20
0.8995600360275502
0.9 5 -20 -60
0.9201136167393691
0.9 5 -20 -20
0.9009669572937176
0.9 5 -20 20
0.914726784020076
0.9 5 20 -60
0.8857310330475751
0.9 5 20 -20
0.9173477969976053
0.9 5 20 20
0.9449578068462906
0.9 6 -60 -60
0.920014813997575
0.9 6 -60 -20
0.9293046914882179
0.9 6 -60 20
0.8947482565305048
0.9 6 -20 -60
0.92097419194445
0.9 6 -20 -20
0.9095337488477747
0.9 6 -20 20
0.9142078290577185
0.9 6 20 -60
0.8950162420003874
0.9 6 20 -20
0.9155921658869326
0.9 6 20 20
0.9396917761874934
0.9 7 -60 -60
0.9305069523817929
0.9 7 -60 -20
0.9538666547578024
0.9 7 -60 20
0.8957667782995612
0.9 7 -20 -60
0.907727974150702
0.9 7 -20 -20
0.9182786114189987
0.9 7 -20 20
0.9035064178929819
0.9 7 20 -60
0.9265113146648902
0.9 7 20 -20
0.9133696258020402
0.9 7 20 20
0.9260024852309215
0.9 8 -60 -60
0.9430

In [22]:
c_curv_fig4 = np.zeros((16, 10, 3, 3))



#first, vary c
for c_, c in enumerate(range(10, 26)):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), c=10, contact_incib_tcresc=c)
                try:
                    curv = measure_curv("c_varying_fig4/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    c_curv_fig4[c_][r_][e_][a_] = curv
                    print(c, r, elev, azim)
                except:
                    name = "c_varying_fig4/"+str(c)+"_"+str(r)
                    occupations = np.load("c_varying_fig4/"+str(c)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("c_varying_fig4/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    c_curv_fig4[c_][r_][e_][a_] = curv
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_c_fig4.npy", c_curv_fig4)

0.9231564268740512
10 0 -60 -60
0.9316380825063791
10 0 -60 -20
0.931904600620774
10 0 -60 20
0.923499155871771
10 0 -20 -60
0.891300640610147
10 0 -20 -20
0.9126182775907342
10 0 -20 20
0.9089841712962904
10 0 20 -60
0.9171495245618095
10 0 20 -20
0.9169536842473941
10 0 20 20
0.9419980393973537
10 1 -60 -60
0.9314448509756941
10 1 -60 -20
0.9254451786694208
10 1 -60 20
0.9338319133144422
10 1 -20 -60
0.920384804352011
10 1 -20 -20
0.9327558579584926
10 1 -20 20
0.9010169292230051
10 1 20 -60
0.9373487384781701
10 1 20 -20
0.9214636636425151
10 1 20 20
0.9348411763888167
10 2 -60 -60
0.9475696843150742
10 2 -60 -20
0.9506353723625599
10 2 -60 20
0.9053932153909672
10 2 -20 -60
0.9135256092740507
10 2 -20 -20
0.9286717660573057
10 2 -20 20
0.8939616577611743
10 2 20 -60
0.9097628471051211
10 2 20 -20
0.9075688148422164
10 2 20 20
0.9341306098221172
10 3 -60 -60
0.920014058584776
10 3 -60 -20
0.9326969800418364
10 3 -60 20
0.9096966589733421
10 3 -20 -60
0.9178125271098139
10 3 -20 -20


0.9279051200958496
12 9 -60 -20
0.9324790839559971
12 9 -60 20
0.9033005026644728
12 9 -20 -60
0.9142616995680637
12 9 -20 -20
0.9115897990479683
12 9 -20 20
0.9155581965249732
12 9 20 -60
0.9082327379203283
12 9 20 -20
0.904980327271278
12 9 20 20
Repeat 9 completed.
0.9247332151784612
13 0 -60 -60
0.9053393920708285
13 0 -60 -20
0.9133211207187143
13 0 -60 20
0.894632828396308
13 0 -20 -60
0.9065551647713767
13 0 -20 -20
0.9073318318520952
13 0 -20 20
0.907933691500724
13 0 20 -60
0.9179063437234031
13 0 20 -20
0.897505008430985
13 0 20 20
0.9407083114817167
13 1 -60 -60
0.930082291896842
13 1 -60 -20
0.9322352804684042
13 1 -60 20
0.911289238890926
13 1 -20 -60
0.9277276832551893
13 1 -20 -20
0.9236809687824972
13 1 -20 20
0.910101129461172
13 1 20 -60
0.9076433093316993
13 1 20 -20
0.9120471166955426
13 1 20 20
0.9441543886267704
13 2 -60 -60
0.9455740816108624
13 2 -60 -20
0.9339380757815635
13 2 -60 20
0.9173837873369726
13 2 -20 -60
0.9237540588733257
13 2 -20 -20
0.932435836921

0.9139553476468422
15 8 -60 -20
0.9339600163568115
15 8 -60 20
0.9017462056748854
15 8 -20 -60
0.9143071952682953
15 8 -20 -20
0.9144058626695651
15 8 -20 20
0.9259706140338889
15 8 20 -60
0.9292860914052692
15 8 20 -20
0.9242133960136804
15 8 20 20
0.9448308262437891
15 9 -60 -60
0.9390535673487159
15 9 -60 -20
0.9304565800267639
15 9 -60 20
0.9061014170616027
15 9 -20 -60
0.9245828289383636
15 9 -20 -20
0.9255724212214103
15 9 -20 20
0.9169806309779165
15 9 20 -60
0.9208279174854027
15 9 20 -20
0.9191732505359087
15 9 20 20
Repeat 9 completed.
0.9470503908549833
16 0 -60 -60
0.9461804783668847
16 0 -60 -20
0.9347248496076918
16 0 -60 20
0.917907524853056
16 0 -20 -60
0.9159298932976238
16 0 -20 -20
0.9143365975526594
16 0 -20 20
0.9168237281689582
16 0 20 -60
0.9229809629047174
16 0 20 -20
0.9290394480675767
16 0 20 20
0.9460255108540148
16 1 -60 -60
0.9392813855881056
16 1 -60 -20
0.9316157582960025
16 1 -60 20
0.8885626887407159
16 1 -20 -60
0.9124492711524069
16 1 -20 -20
0.920372

0.9204611146352595
18 7 -60 20
0.9053125867237087
18 7 -20 -60
0.9093542748164926
18 7 -20 -20
0.8787250860643032
18 7 -20 20
0.9016717175937924
18 7 20 -60
0.910549915058191
18 7 20 -20
0.9080924467781303
18 7 20 20
0.9501464977181268
18 8 -60 -60
0.9412068741289732
18 8 -60 -20
0.9308043327640778
18 8 -60 20
0.9066478849292516
18 8 -20 -60
0.899889083921852
18 8 -20 -20
0.9142095405781352
18 8 -20 20
0.89084163739292
18 8 20 -60
0.9167579988990198
18 8 20 -20
0.8950223492650135
18 8 20 20
0.9362805759070006
18 9 -60 -60
0.9379666963865759
18 9 -60 -20
0.9438070765558715
18 9 -60 20
0.9020306778825953
18 9 -20 -60
0.9063736327818599
18 9 -20 -20
0.9226424181512151
18 9 -20 20
0.9063092767874353
18 9 20 -60
0.9017682718406599
18 9 20 -20
0.9094031379838672
18 9 20 20
Repeat 9 completed.
0.9347359782301731
19 0 -60 -60
0.93627146070226
19 0 -60 -20
0.9408358061643385
19 0 -60 20
0.9048372143142214
19 0 -20 -60
0.9275252396170675
19 0 -20 -20
0.9260912807065118
19 0 -20 20
0.913794335121

0.91722664607332
21 6 -20 -60
0.9145400011242804
21 6 -20 -20
0.8960072126688239
21 6 -20 20
0.9115741008624696
21 6 20 -60
0.9139230800102632
21 6 20 -20
0.9135749515646403
21 6 20 20
0.9439717455729323
21 7 -60 -60
0.9437726522799408
21 7 -60 -20
0.9322780166928262
21 7 -60 20
0.8916740293429608
21 7 -20 -60
0.9038268869332813
21 7 -20 -20
0.9250701540622872
21 7 -20 20
0.9166027177142982
21 7 20 -60
0.9248400355028787
21 7 20 -20
0.9173629420710046
21 7 20 20
0.932102619967803
21 8 -60 -60
0.9330255485820945
21 8 -60 -20
0.931225394180407
21 8 -60 20
0.9168476400926625
21 8 -20 -60
0.9043851943923444
21 8 -20 -20
0.9084694721096743
21 8 -20 20
0.9231530014719666
21 8 20 -60
0.9143083027442159
21 8 20 -20
0.918563939230146
21 8 20 20
0.9376541956370039
21 9 -60 -60
0.9249933979400708
21 9 -60 -20
0.9280726241546431
21 9 -60 20
0.9115907464450272
21 9 -20 -60
0.9175276512011098
21 9 -20 -20
0.9178964872701202
21 9 -20 20
0.9130299272408087
21 9 20 -60
0.9152515378531054
21 9 20 -20
0.

0.9105062505543835
24 5 -20 -60
0.9232935329807272
24 5 -20 -20
0.9270773409023962
24 5 -20 20
0.9169437421651782
24 5 20 -60
0.9101482263691715
24 5 20 -20
0.9293324235379812
24 5 20 20
0.9324595208619036
24 6 -60 -60
0.9389410645106365
24 6 -60 -20
0.940056084576903
24 6 -60 20
0.9191882019186588
24 6 -20 -60
0.9191449793373156
24 6 -20 -20
0.9096455831954635
24 6 -20 20
0.9214028236521664
24 6 20 -60
0.900897521021342
24 6 20 -20
0.9204375426674167
24 6 20 20
0.9451162797414858
24 7 -60 -60
0.9258522979756193
24 7 -60 -20
0.9327734650139107
24 7 -60 20
0.9087772334751221
24 7 -20 -60
0.9211760110210758
24 7 -20 -20
0.9009491051996231
24 7 -20 20
0.9024797337948207
24 7 20 -60
0.9055744352280586
24 7 20 -20
0.9049269034210795
24 7 20 20
0.935001144270474
24 8 -60 -60
0.9425704837833967
24 8 -60 -20
0.9407274480377927
24 8 -60 20
0.9004029249326483
24 8 -20 -60
0.9031161551943752
24 8 -20 -20
0.8934658050678379
24 8 -20 20
0.9096245556115725
24 8 20 -60
0.9054635667169725
24 8 20 -20


In [23]:
q_curv_fig5 = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, q in enumerate([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), q=10, contact_inqib_tqresq=c)
                try:
                    curv = measure_curv("q_varying_fig5/"+str(q)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    q_curv_fig5[c_][r_][e_][a_] = curv
                    print(q, r, elev, azim)
                except:
                    name = "q_varying_fig5/"+str(q)+"_"+str(r)
                    occupations = np.load("q_varying_fig5/"+str(q)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("q_varying_fig5/"+str(q)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    q_curv_fig5[c_][r_][e_][a_] = curv
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_q_fig5.npy", q_curv_fig5)

0.9304562512359584
0.0 0 -60 -60
0.9064193824434544
0.0 0 -60 -20
0.9227303451398539
0.0 0 -60 20
0.905005371889823
0.0 0 -20 -60
0.884884060081963
0.0 0 -20 -20
0.8901063457145393
0.0 0 -20 20
0.8995860790811777
0.0 0 20 -60
0.9028762564108076
0.0 0 20 -20
0.9161497760052447
0.0 0 20 20
0.9356396425040556
0.0 1 -60 -60
0.9249912819097742
0.0 1 -60 -20
0.9239487531282465
0.0 1 -60 20
0.9068897938570935
0.0 1 -20 -60
0.9077577938346675
0.0 1 -20 -20
0.9096324047057078
0.0 1 -20 20
0.9306581063668865
0.0 1 20 -60
0.9191981208695624
0.0 1 20 -20
0.9110387769052047
0.0 1 20 20
0.9308487570162294
0.0 2 -60 -60
0.9422384913424026
0.0 2 -60 -20
0.9235552044110406
0.0 2 -60 20
0.9204227072044071
0.0 2 -20 -60
0.9043292873500874
0.0 2 -20 -20
0.9127118676113053
0.0 2 -20 20
0.9075399612547329
0.0 2 20 -60
0.917348480893285
0.0 2 20 -20
0.9193988693910504
0.0 2 20 20
0.9301598482384685
0.0 3 -60 -60
0.9343468712536793
0.0 3 -60 -20
0.9182599191498564
0.0 3 -60 20
0.9027758174441624
0.0 3 -20 -60

0.9115643205032441
0.2 8 -20 -60
0.9179835569270448
0.2 8 -20 -20
0.912916659500262
0.2 8 -20 20
0.9124269310169726
0.2 8 20 -60
0.9191392960977651
0.2 8 20 -20
0.922546227050354
0.2 8 20 20
0.9178472440116121
0.2 9 -60 -60
0.9347915458010995
0.2 9 -60 -20
0.9306257642163054
0.2 9 -60 20
0.9247743959145212
0.2 9 -20 -60
0.9034413923955066
0.2 9 -20 -20
0.9059125524996673
0.2 9 -20 20
0.9054303970445762
0.2 9 20 -60
0.9070649854829701
0.2 9 20 -20
0.9060982923747529
0.2 9 20 20
Repeat 9 completed.
0.9293096932817826
0.3 0 -60 -60
0.934092956229906
0.3 0 -60 -20
0.9189257948620246
0.3 0 -60 20
0.8976216367590546
0.3 0 -20 -60
0.9178917065621213
0.3 0 -20 -20
0.9118338328155192
0.3 0 -20 20
0.9148048115870321
0.3 0 20 -60
0.917109368243582
0.3 0 20 -20
0.9172334119485925
0.3 0 20 20
0.9320816367431838
0.3 1 -60 -60
0.9248991817448513
0.3 1 -60 -20
0.9443786938551237
0.3 1 -60 20
0.9224181229747267
0.3 1 -20 -60
0.914872060803732
0.3 1 -20 -20
0.9299113900147472
0.3 1 -20 20
0.899700500134

0.9016460339810026
0.5 6 -20 20
0.9127360806006533
0.5 6 20 -60
0.9057762749359727
0.5 6 20 -20
0.904323366327568
0.5 6 20 20
0.9367800851947445
0.5 7 -60 -60
0.9294517543609178
0.5 7 -60 -20
0.922478599477142
0.5 7 -60 20
0.9007054872705414
0.5 7 -20 -60
0.9011384520767445
0.5 7 -20 -20
0.8858567774048046
0.5 7 -20 20
0.8992325306463509
0.5 7 20 -60
0.892138097242897
0.5 7 20 -20
0.8852053243668997
0.5 7 20 20
0.9335382518036075
0.5 8 -60 -60
0.9300789665930531
0.5 8 -60 -20
0.9200009855771768
0.5 8 -60 20
0.9208458521604768
0.5 8 -20 -60
0.9156278649793436
0.5 8 -20 -20
0.9204902222973487
0.5 8 -20 20
0.9160165338555266
0.5 8 20 -60
0.9097374867568429
0.5 8 20 -20
0.9069620931818434
0.5 8 20 20
0.9307408348706896
0.5 9 -60 -60
0.9364969951160724
0.5 9 -60 -20
0.9279679787513652
0.5 9 -60 20
0.9168524064660645
0.5 9 -20 -60
0.9027381194451132
0.5 9 -20 -20
0.9000765387104178
0.5 9 -20 20
0.9299484957434588
0.5 9 20 -60
0.9108200385001024
0.5 9 20 -20
0.9146320005655744
0.5 9 20 20
Rep

0.8831533440973656
0.8 4 20 -60
0.930919614859142
0.8 4 20 -20
0.9038985170018591
0.8 4 20 20
0.9237131902473072
0.8 5 -60 -60
0.9267838678894779
0.8 5 -60 -20
0.9246407448936859
0.8 5 -60 20
0.8874085090210825
0.8 5 -20 -60
0.9059517891332667
0.8 5 -20 -20
0.883983800544005
0.8 5 -20 20
0.8919434550951116
0.8 5 20 -60
0.9285050382447169
0.8 5 20 -20
0.9145825648133473
0.8 5 20 20
0.9333421945354948
0.8 6 -60 -60
0.9392725777983039
0.8 6 -60 -20
0.9232652501654102
0.8 6 -60 20
0.9027124590684857
0.8 6 -20 -60
0.9191353018773043
0.8 6 -20 -20
0.9062021019732471
0.8 6 -20 20
0.9125454728223429
0.8 6 20 -60
0.9117330190392443
0.8 6 20 -20
0.9039613058264852
0.8 6 20 20
0.9179006760337363
0.8 7 -60 -60
0.9088639432210994
0.8 7 -60 -20
0.9259954288817269
0.8 7 -60 20
0.8736024724698831
0.8 7 -20 -60
0.9017282660010775
0.8 7 -20 -20
0.9419390396503997
0.8 7 -20 20
0.8799677192696996
0.8 7 20 -60
0.9117777036454141
0.8 7 20 -20
0.901653581056898
0.8 7 20 20
0.937595176968075
0.8 8 -60 -60
0.9

In [9]:
beta_curv_fig4 = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, beta in enumerate(range(1, 11)):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inbetaib_tbetaresbeta=c)
                try:
                    curv = measure_curv("beta_varying_fig4/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    beta_curv_fig4[c_][r_][e_][a_] = curv
                    print(beta, r, elev, azim)
                except:
                    name = "beta_varying_fig4/"+str(beta)+"_"+str(r)
                    occupations = np.load("beta_varying_fig4/"+str(beta)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("beta_varying_fig4/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    beta_curv_fig4[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.patbeta.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_beta_fig4.npy", beta_curv_fig4)

0.9446768263590122
1 0 -60 -60
0.9469003309871968
1 0 -60 -20
0.9468280866730203
1 0 -60 20
0.9011032325123485
1 0 -20 -60
0.9209102840838446
1 0 -20 -20
0.9058945521911458
1 0 -20 20
0.9017905005446192
1 0 20 -60
0.8941604507634614
1 0 20 -20
0.9027459398272957
1 0 20 20
0.9217106009998094
1 1 -60 -60
0.9119226905010656
1 1 -60 -20
0.8958564931813872
1 1 -60 20
0.8999077147909699
1 1 -20 -60
0.90136296225511
1 1 -20 -20
0.9055436795079165
1 1 -20 20
0.8811590923088483
1 1 20 -60
0.9080291928290595
1 1 20 -20
0.9048616338931257
1 1 20 20
0.9448653377928923
1 2 -60 -60
0.9336299319942221
1 2 -60 -20
0.9494889275688042
1 2 -60 20
0.8864533993018292
1 2 -20 -60
0.8941730132880825
1 2 -20 -20
0.902011040696706
1 2 -20 20
0.895833802674177
1 2 20 -60
0.9039755002357405
1 2 20 -20
0.910670812263278
1 2 20 20
0.923326573033953
1 3 -60 -60
0.9285661410402808
1 3 -60 -20
0.9241256966095887
1 3 -60 20
0.9066655140037533
1 3 -20 -60
0.9089554032370726
1 3 -20 -20
0.908331727478719
1 3 -20 20
0.91

0.92466980141385
4 0 -60 -60
0.9197285120155321
4 0 -60 -20
0.9265046441891753
4 0 -60 20
0.9117574534392207
4 0 -20 -60
0.9100316886752489
4 0 -20 -20
0.9086360881137218
4 0 -20 20
0.9037492638687535
4 0 20 -60
0.92517170963417
4 0 20 -20
0.9039733142561058
4 0 20 20
0.9174141089442578
4 1 -60 -60
0.9173593716026618
4 1 -60 -20
0.9099182233075986
4 1 -60 20
0.9203023354341192
4 1 -20 -60
0.9223668769023927
4 1 -20 -20
0.9042472782519213
4 1 -20 20
0.923469067011428
4 1 20 -60
0.9250606617321596
4 1 20 -20
0.9362735019778274
4 1 20 20
0.9349020878353422
4 2 -60 -60
0.9265756548068093
4 2 -60 -20
0.9256873536267295
4 2 -60 20
0.9052028435053908
4 2 -20 -60
0.9215888199374626
4 2 -20 -20
0.9081802296419957
4 2 -20 20
0.9023546550705345
4 2 20 -60
0.921937716380317
4 2 20 -20
0.9138658270290864
4 2 20 20
0.940624910043456
4 3 -60 -60
0.9280126589398981
4 3 -60 -20
0.9301112999903062
4 3 -60 20
0.9164667589945191
4 3 -20 -60
0.9034155725401912
4 3 -20 -20
0.9070252699476521
4 3 -20 20
0.90

0.9180130321514947
7 0 -60 -60
0.9197459750221059
7 0 -60 -20
0.9205190790290202
7 0 -60 20
0.9176537962965582
7 0 -20 -60
0.9163702082377351
7 0 -20 -20
0.9179048644947225
7 0 -20 20
0.8977204707329897
7 0 20 -60
0.9092192733365672
7 0 20 -20
0.9173704806003635
7 0 20 20
0.9206471481913681
7 1 -60 -60
0.9285009055953326
7 1 -60 -20
0.9348005563998026
7 1 -60 20
0.9120805183650472
7 1 -20 -60
0.9022889023493633
7 1 -20 -20
0.9289439961249348
7 1 -20 20
0.9173820136800261
7 1 20 -60
0.9189422911018759
7 1 20 -20
0.9316189517513181
7 1 20 20
0.916155978900841
7 2 -60 -60
0.9312425895273405
7 2 -60 -20
0.9365467262684929
7 2 -60 20
0.9078178953091344
7 2 -20 -60
0.9206880053485043
7 2 -20 -20
0.9216630388864061
7 2 -20 20
0.8984205519250202
7 2 20 -60
0.9214491380159054
7 2 20 -20
0.924179878358131
7 2 20 20
0.9229154109783865
7 3 -60 -60
0.949345464431123
7 3 -60 -20
0.9237402207832662
7 3 -60 20
0.9137796297149198
7 3 -20 -60
0.9321898890253071
7 3 -20 -20
0.8965200955378686
7 3 -20 20


0.9532542812885323
10 0 -60 -60
0.944100983706355
10 0 -60 -20
0.935764016130462
10 0 -60 20
0.8956045815542195
10 0 -20 -60
0.9058075498164236
10 0 -20 -20
0.9141372596896032
10 0 -20 20
0.8995893977347009
10 0 20 -60
0.9150330264970051
10 0 20 -20
0.9210822640665168
10 0 20 20
0.9393805864743257
10 1 -60 -60
0.926648870070456
10 1 -60 -20
0.9311063317744619
10 1 -60 20
0.9238938586227552
10 1 -20 -60
0.9142148315696159
10 1 -20 -20
0.9107555077277782
10 1 -20 20
0.899352091377543
10 1 20 -60
0.9212033559171865
10 1 20 -20
0.9176401071424501
10 1 20 20
0.9348502991954218
10 2 -60 -60
0.9239954716017393
10 2 -60 -20
0.9344454805606974
10 2 -60 20
0.9070031719270241
10 2 -20 -60
0.9219411120819746
10 2 -20 -20
0.9270981787970884
10 2 -20 20
0.8959324728044206
10 2 20 -60
0.9072375824110323
10 2 20 -20
0.9253891494542348
10 2 20 20
0.921408045508369
10 3 -60 -60
0.9322817268929421
10 3 -60 -20
0.9324191532364859
10 3 -60 20
0.897276390410893
10 3 -20 -60
0.9348755852813598
10 3 -20 -20
0

In [11]:
q_curv_fig6 = np.zeros((10, 10, 3, 3))



#first, vary c
for c_, q in enumerate([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), q=10, contact_inqib_tqresq=c)
                try:
                    curv = measure_curv("q_varying_fig6/"+str(q)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    q_curv_fig6[c_][r_][e_][a_] = curv
                    print(q, r, elev, azim)
                except:
                    name = "q_varying_fig6/"+str(q)+"_"+str(r)
                    occupations = np.load("q_varying_fig6/"+str(q)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("q_varying_fig6/"+str(q)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    q_curv_fig6[c_][r_][e_][a_] = curv
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_q_fig6.npy", q_curv_fig6)

0.9333330843407384
0.0 0 -60 -60
0.936746564357167
0.0 0 -60 -20
0.9302485432549034
0.0 0 -60 20
0.8813971409372116
0.0 0 -20 -60
0.8793814201952376
0.0 0 -20 -20
0.9111932329289926
0.0 0 -20 20
0.8985611047912129
0.0 0 20 -60
0.8948376096609348
0.0 0 20 -20
0.899056057768906
0.0 0 20 20
0.9254632697520385
0.0 1 -60 -60
0.9139353857587528
0.0 1 -60 -20
0.9288579011915495
0.0 1 -60 20
0.9125508141521904
0.0 1 -20 -60
0.9081344260039057
0.0 1 -20 -20
0.8920208016541628
0.0 1 -20 20
0.8948561752029608
0.0 1 20 -60
0.895256727767655
0.0 1 20 -20
0.8894934508217995
0.0 1 20 20
0.9295954058996746
0.0 2 -60 -60
0.9355678354026635
0.0 2 -60 -20
0.9247305342959647
0.0 2 -60 20
0.8955639541074585
0.0 2 -20 -60
0.9000533221871813
0.0 2 -20 -20
0.8865608523458272
0.0 2 -20 20
0.9067769991908389
0.0 2 20 -60
0.8698613915270434
0.0 2 20 -20
0.9171388339914391
0.0 2 20 20
0.9294359014828084
0.0 3 -60 -60
0.8979365793950441
0.0 3 -60 -20
0.9148134411724784
0.0 3 -60 20
0.9093022018881705
0.0 3 -20 -60

0.8939842649148886
0.2 8 -20 -60
0.884103542368855
0.2 8 -20 -20
0.8897777256353645
0.2 8 -20 20
0.9110910040085511
0.2 8 20 -60
0.8960267408459932
0.2 8 20 -20
0.9005173712981555
0.2 8 20 20
0.9107409946335937
0.2 9 -60 -60
0.9289902267075372
0.2 9 -60 -20
0.913910498952099
0.2 9 -60 20
0.8983883281285238
0.2 9 -20 -60
0.8812822117001318
0.2 9 -20 -20
0.8980775678065445
0.2 9 -20 20
0.9146070115586493
0.2 9 20 -60
0.8678869681276947
0.2 9 20 -20
0.8881638016066037
0.2 9 20 20
Repeat 9 completed.
0.9341481239392266
0.3 0 -60 -60
0.9339952182963557
0.3 0 -60 -20
0.9238674844814906
0.3 0 -60 20
0.88797356589307
0.3 0 -20 -60
0.9113469200697055
0.3 0 -20 -20
0.9028892158863481
0.3 0 -20 20
0.9011187464608547
0.3 0 20 -60
0.8938675913162686
0.3 0 20 -20
0.8835297274843897
0.3 0 20 20
0.9226289607045098
0.3 1 -60 -60
0.9180752574945223
0.3 1 -60 -20
0.9215243283180948
0.3 1 -60 20
0.9106977627196703
0.3 1 -20 -60
0.8884589280012546
0.3 1 -20 -20
0.8998256640442603
0.3 1 -20 20
0.87849547838

0.9064828906217127
0.5 6 -20 -20
0.8824915005146196
0.5 6 -20 20
0.9200848177167017
0.5 6 20 -60
0.9247446178292146
0.5 6 20 -20
0.8980501931248639
0.5 6 20 20
0.9267423873274141
0.5 7 -60 -60
0.9022856550203356
0.5 7 -60 -20
0.9078929129516656
0.5 7 -60 20
0.9090278093900527
0.5 7 -20 -60
0.8978243760426917
0.5 7 -20 -20
0.9167027573430687
0.5 7 -20 20
0.8861694106751266
0.5 7 20 -60
0.8638889804640808
0.5 7 20 -20
0.8944143267004928
0.5 7 20 20
0.8962610349605374
0.5 8 -60 -60
0.8689005921954628
0.5 8 -60 -20
0.9003612151038579
0.5 8 -60 20
0.9172292118750439
0.5 8 -20 -60
0.8808643009895484
0.5 8 -20 -20
0.8785080805830349
0.5 8 -20 20
0.9173073641326213
0.5 8 20 -60
0.8725886622729215
0.5 8 20 -20
0.8815595804202057
0.5 8 20 20
0.9374975779031386
0.5 9 -60 -60
0.9184445176121355
0.5 9 -60 -20
0.9358882615433894
0.5 9 -60 20
0.8918002049307349
0.5 9 -20 -60
0.8957656575098972
0.5 9 -20 -20
0.8829863639929948
0.5 9 -20 20
0.90868486523727
0.5 9 20 -60
0.906229649401617
0.5 9 20 -20
0

0.8784662195796138
0.8 4 20 -60
0.8702653831292526
0.8 4 20 -20
0.8728717284056342
0.8 4 20 20
0.9145124611928843
0.8 5 -60 -60
0.8985814181396755
0.8 5 -60 -20
0.9223156707196509
0.8 5 -60 20
0.8962983848097683
0.8 5 -20 -60
0.8995116035442856
0.8 5 -20 -20
0.8486360150697639
0.8 5 -20 20
0.8699369998952585
0.8 5 20 -60
0.8980934949563147
0.8 5 20 -20
0.8923115028168204
0.8 5 20 20
0.8651090530959137
0.8 6 -60 -60
0.8390773191547372
0.8 6 -60 -20
0.8246229550818781
0.8 6 -60 20
0.8717473505042208
0.8 6 -20 -60
0.849903067511114
0.8 6 -20 -20
0.8450150963800497
0.8 6 -20 20
0.860978339757727
0.8 6 20 -60
0.8537830941421685
0.8 6 20 -20
0.8624001662622744
0.8 6 20 20
0.8838446116690727
0.8 7 -60 -60
0.8655798096576092
0.8 7 -60 -20
0.9014521941180066
0.8 7 -60 20
0.8680773149222764
0.8 7 -20 -60
0.8596706339139047
0.8 7 -20 -20
0.8379613106052878
0.8 7 -20 20
0.8370908878046494
0.8 7 20 -60
0.8597981673819323
0.8 7 20 -20
0.8574712453002594
0.8 7 20 20
0.8949647684860329
0.8 8 -60 -60
0